# Spark Introduction

## Distributed computing
Idea: split computational load across 2+ computers (nodes)  
Master <-> workers architecture  

### Pros
1. Speeds up computation
2. Spread out the risk of a crash

### Cons
1. Distributing data is expensive (network bottleneck)
2. Synchronization is expensive (disk bottleneck)
3. Writing distributed algorithms is difficult

## MapReduce Model
Both a computational model and a specific tool name

Stages:  
1. Input
2. Split - move part of data to each worker
3. Map - apply local transform to data on each node
4. Shuffle - Exchange data between nodes as needed
5. Reduce - Compute aggrate result of some kind
6. Output

Example: counting unique words in document  
+ input: document with all words
+ split: divide document evenly among nodes
+ map: count unique words on each node
+ shuffle: group unique words on each node, i.e. first node keeps all counts of word "car"
+ reduce: take sum of each unique word
+ output: dict of word counts

In spark: specify map and reduce function and spark will handle the rest.  

## No Cluster for Me?
Spark can still be useful for running multi core workloads

## Spark
+ Written in Scala
+ Invented at Berkeley
+ Open source via Apache
+ Default big-data tool in industry

### Features
+ Built-in resilience - re-assigns jobs that don't return results
+ Node awareness
+ Supports tons of file formats/sources
+ Streaming
+ Many platforms
+ high-level api

## Differences from Pandas
+ Immutable dataframes
+ No random access
+ Automatic optimizatioin

## High Level API
2 kinds of operations
+ transformation: like torch.view - no computation is done, just add a transform
+ action: kicks off computation - applies transforms to data etc.

## Optimization
+ Data often goes thgouth many transformations before hitting an action.  
+ Spark Catalyst engine optimizes order of operations to be faster - filter as early as possible

## No Random Access
+ Master knows how large split to each node is, but doesn't remember exactly what data is where
+ Can't get a random row off a random node, that would take forever
+ Order not guaranteed

In [1]:
from pyspark.sql import *
from pyspark.sql import functions as f
from pyspark.sql.types import *

spark = SparkSession.builder.appName("SparkIntro").getOrCreate()

In [13]:
df = spark.read.option('header',True).option('inferSchema',True).csv('data/titanic.csv')

In [18]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|   false|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|    true|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|    true|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|    true|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|   false|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|   false|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [34]:
%%time
males = df.filter(f.col('Sex') == 'male')
num_males = males.count()
num_males

CPU times: user 1.68 ms, sys: 248 µs, total: 1.93 ms
Wall time: 69.8 ms


In [24]:
import pandas as pd

In [25]:
pdf = df.toPandas()

In [33]:
%%time
males = pdf[pdf['Sex'] == 'male']
males['Sex'].count()

CPU times: user 1.49 ms, sys: 219 µs, total: 1.71 ms
Wall time: 1.18 ms
